In [1]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row, SparkSession

In [2]:
spark = SparkSession.builder.appName('Recommender-app').getOrCreate()

22/02/01 18:56:29 WARN Utils: Your hostname, oussama-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.15 instead (on interface wlp3s0)
22/02/01 18:56:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/01 18:56:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

schema = StructType([
    StructField("book_id", IntegerType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("rating", FloatType(), True)])


ratings = spark.read.csv("../data/books_ratings.csv", header=True, schema=schema)

In [6]:
ratings.show()

+-------+-------+------+
|book_id|user_id|rating|
+-------+-------+------+
|      1|    314|   5.0|
|      1|    439|   3.0|
|      1|    588|   5.0|
|      1|   1169|   4.0|
|      1|   1185|   4.0|
|      1|   2077|   4.0|
|      1|   2487|   4.0|
|      1|   2900|   5.0|
|      1|   3662|   4.0|
|      1|   3922|   5.0|
|      1|   5379|   5.0|
|      1|   5461|   3.0|
|      1|   5885|   5.0|
|      1|   6630|   5.0|
|      1|   7563|   3.0|
|      1|   9246|   1.0|
|      1|  10140|   4.0|
|      1|  10146|   5.0|
|      1|  10246|   4.0|
|      1|  10335|   4.0|
+-------+-------+------+
only showing top 20 rows



In [18]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id", itemCol="book_id", ratingCol="rating",
          coldStartStrategy="drop")
model = als.fit(ratings)

In [6]:
# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))

22/02/01 13:04:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/oussama/Desktop/workspace/matrix_factorization_recommender/data/books_ratings.csv


Root-mean-square error = 2.225046600971715


In [7]:
# movieRecs = model.recommendForAllItems(1)

/home/oussama/.local/share/virtualenvs/matrix_factorization_recommender-qyAKTrSQ/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [8]:
# movieRecs.show()

+-------+-------------------+
|book_id|    recommendations|
+-------+-------------------+
|      1|         [{0, 0.0}]|
|     12|[{1775, 5.1165895}]|
|     13|  [{58, 10.170677}]|
|     22| [{545, 3.8608067}]|
|     26| [{329, 5.6786933}]|
|     27|         [{0, 0.0}]|
|     28|  [{58, 12.186274}]|
|     31|  [{58, 7.4835396}]|
|     34|         [{0, 0.0}]|
|     44|         [{0, 0.0}]|
|     47|  [{181, 8.949086}]|
|     52|[{1249, 4.8477545}]|
|     53|[{1760, 14.496603}]|
|     65|         [{0, 0.0}]|
|     76|         [{0, 0.0}]|
|     78|   [{509, 9.35182}]|
|     81|         [{0, 0.0}]|
|     85| [{545, 6.4346776}]|
|     91|         [{0, 0.0}]|
|     93|[{1111, 10.883413}]|
+-------+-------------------+
only showing top 20 rows



In [19]:
import pandas as pd

df_user_0 = pd.DataFrame([0], columns=['user_id'])

In [20]:
df_user_0.head()

,user_id
0,0


In [23]:
user_0_ratings = model.recommendForUserSubset(spark.createDataFrame(pd.DataFrame([0], columns=['user_id'])), 10)

/home/oussama/.local/share/virtualenvs/matrix_factorization_recommender-qyAKTrSQ/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
from pyspark.sql.functions import explode

user_0_ratings.select("user_id", explode("recommendations")
  .alias("recommendation")).select("user_id", "recommendation.*").drop('user_id').toPandas()

In [111]:
user_0_ratings.select("recommendations.book_id", "recommendations.rating").first().rating[0]

11.715402603149414

In [12]:
ratings.select('book_id').dropDuplicates().count()

302773